# Getting Started

## Kernel functions

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from lib.mesh import Complex,Mesh
from lib.materials import Physics,PDE,Domain,Stern,Robin,Dirichlet
from lib.assembly import StaticSolution,Solution

## List of variables

In [2]:
# This section will load user inputs and set global variables
# Not implemented yet
# Go to user inputs for desired test case as a temporary solution

# Placeholder
# Need to consider which variables are user inputs

# E4D files
# 1. file .inp
# 2. file .sig
# 3. file .cfg
# 4. file .inv (skip for sip)
# 5. file .out
# 6. file .srv

# Possible SIP files
# 1. file .inp (specifies run_mode: 1 for mesh generation; 2 for static solution; 3 for coupled solution)
# 2. file .hdf5 (stores mesh and material properties for domain equation and boundary equation)
# 3. file .cfg (will generate .poly for triangle; will be compatible with sphere/slab/membrane models)
# 4. file .out (specifies output options)
# 5. file .srv (specifies survey/external field options)

# print('='*80)
# %whos module
# print('='*80)
# %whos type
# print('='*80)
# %whos str
# print('='*80)
# %whos int
# print('='*80)
# %whos float
# print('='*80)
# %whos list
# print('='*80)
# %whos dict
# print('='*80)
# %whos ndarray matrix
# print('='*80)
# %whos function
%whos

Variable         Type      Data/Info
------------------------------------
Complex          type      <class 'lib.mesh.Complex'>
Dirichlet        type      <class 'lib.materials.Dirichlet'>
Domain           type      <class 'lib.materials.Domain'>
Mesh             type      <class 'lib.mesh.Mesh'>
PDE              type      <class 'lib.materials.PDE'>
Physics          type      <class 'lib.materials.Physics'>
Robin            type      <class 'lib.materials.Robin'>
Solution         type      <class 'lib.assembly.Solution'>
StaticSolution   type      <class 'lib.assembly.StaticSolution'>
Stern            type      <class 'lib.materials.Stern'>
np               module    <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>


# Example 1

## User inputs

In [3]:
def build_f(x,y,*args):
    dist = np.sqrt(x**2+y**2)
    f = np.zeros_like(dist)
    mask = dist>0
    f[mask] = np.pi/2.0*(1/dist[mask]*np.sin(np.pi*dist[mask]/2.0)
                         +np.pi/2.0*np.cos(np.pi*dist[mask]/2.0))
    f[~mask] = np.pi/2.0*(np.pi/2.0+np.pi/2.0)
    return f

def build_s(x,y,*args):
    dist = np.sqrt(x**2+y**2)
    s = np.cos(np.pi*dist/2.0)
    return s

mesh = Mesh.importer(
    prefix = 'slab_2d',
    triangle = 'triangle',
    axis_symmetry = 'None',
    )

pde = PDE(
    c_x = {'is_inside_domain':[[1.0]]},
    c_y = {'is_inside_domain':[[1.0]]},
    f_n = {'is_on_inside_domain':[build_f]},
    s_n = {'is_on_outer_bound':[build_s]},
    )

print('Done')

Reading slab_2d.1.node
Reading slab_2d.1.ele
Reading slab_2d.1.edge

Computing shape functions of triangular elements
This will take a minute
Time elapsed  0.2950255870819092 sec

Computing shape functions of line segments
This will take a minute
Time elapsed  0.03809380531311035 sec

Elemental PDE system is: 1 by 1

Done


## Solution

In [10]:
domain = Domain(mesh,pde)
stern = Stern(mesh,pde)
robin = Robin(mesh,pde)
dirichlet = Dirichlet(mesh,pde)


gamma_x (14213, 1)
gamma_y (14213, 1)
f (14213, 1)
f_n (7171, 1)
f_d (7171, 1)
gamma_x (14213, 1)
gamma_y (14213, 1)
f (14213, 1)
f_n (7171, 1)
f_d (7171, 1)


# Example 2

In [ ]:
slab = Complex.init_slab(
    radius_air = 1.0, #radius of air
    height_air = 1.0, #height of air
    radius_water = 1.0, #radius of water
    height_water = 0.2, #height of water
    radius_solid = 1.0, #radius of solid
    height_solid = 1.0, #height of solid
    axis_symmetry = 'None', #X, or Y, or None
    )
slab.visualize()

mesh = Mesh.builder(
    prefix = 'slab_test',
    triangle = 'triangle',
    cpts = slab.cpts,
    segs = slab.segs,
    holes = slab.holes,
    zones = slab.zones,
    axis_symmetry = slab.axis_symmetry
    )
mesh.visualize()

phys = Physics(
    c_ion = [1.0,1.0], #ion concentration at infinity [mol/m^3]
    z_ion = [-1.0,1.0], #ion valence or charge number [SI]
    mu_a = [5e-8,5e-8], #ion mobility in electrolyte [m^2/(Vs)]
    mu_s = 5e-9, #ion mobility in solid [m^2/(Vs)]
    rel_perm_a = 80.0, #relative permittivity of electrolyte [SI]
    rel_perm_i = 4.5, #relative permittivity of solid [SI]
    sigma_solid = -0.01, #surface charge density for solid [C]
    temperature = 293.0, #ambient temperature [K]
    e_0 = [0.0,0.0], #Ex/Ey at infinity
    f_0 = [[0.0,0.0,0.0]], #x/y/charge density at point sources
    s_0 = 0.0, #voltage on equipotential surface
    is_solid_metal = False, #True if solid is metal otherwise false
    )

#Overide mesh by import another mesh
mesh = Mesh.importer(
    prefix = 'slab_2d',
    triangle = 'triangle',
    axis_symmetry = slab.axis_symmetry,
    )

mesh.visualize(
    edge_flags=[1,2,3,11,12,14],
    elem_flags=[2,3]
    )

def build_f(x,y,*args):
    dist = np.sqrt(x**2+y**2)
    f = np.zeros_like(dist)
    mask = dist>0
    f[mask] = np.pi/2.0*(1/dist[mask]*np.sin(np.pi*dist[mask]/2.0)
                         +np.pi/2.0*np.cos(np.pi*dist[mask]/2.0))
    f[~mask] = np.pi/2.0*(np.pi/2.0+np.pi/2.0)
    return f

def build_s(x,y,*args):
    dist = np.sqrt(x**2+y**2)
    s = np.cos(np.pi*dist/2.0)
    return s

pde = PDE(
    c_x = {'is_inside_domain':1.0},
    c_y = {'is_inside_domain':1.0},
    f_n = {'is_on_inside_domain':build_f},
    s_n = {'is_on_outer_bound':build_s},
    )

domain = Domain(mesh,pde)
stern = Stern(mesh,pde)
robin = Robin(mesh,pde)
dirichlet = Dirichlet(mesh,pde)

stat = PDE.init_static(phys)
# domain = Domain(mesh,stat)
# stern = Stern(mesh,stat)
# robin = Robin(mesh,stat)
# dirichlet = Dirichlet(mesh,stat)
print('Done')

# Psuedo-code

In [ ]:
run_simulation(model,survey)